In [1]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [2]:
X,y = make_classification(n_samples=10000, n_features=10,n_informative=3)

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [4]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)

print("Decision Tree accuracy",accuracy_score(y_test,y_pred))

Decision Tree accuracy 0.9215


# Bagging

In [6]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.5,  # 50 % data ko liya gya hai 
    bootstrap=True,  # sampling with replacement ho raha hai
    random_state=42
)

In [7]:
bag.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [8]:
y_pred = bag.predict(X_test)

In [9]:
accuracy_score(y_test,y_pred)

0.9475

In [11]:
bag.estimators_samples_[0].shape

(4000,)

In [ ]:
bag.estimators_features_[0].shape

(10,)

# Bagging using SVM

In [13]:
bag = BaggingClassifier(
    estimator=SVC(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    random_state=42
)

In [14]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Bagging using SVM",accuracy_score(y_test,y_pred))

Bagging using SVM 0.937


# Pasting

In [ ]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False,  
    random_state=42,
    verbose = 1,
    n_jobs=-1  # isse mera work fast ho jata hai bcz cpu apne task ko divide karleta hai
)

In [16]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Pasting classifier",accuracy_score(y_test,y_pred))

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    9.0s remaining:   45.5s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    9.5s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    0.1s remaining:    0.7s


Pasting classifier 0.949


[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    0.2s finished


# Random Subspaces
- only column sampling is doing

In [18]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,  # iska matlab pura sample le raha hoon
    bootstrap=False,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [19]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Subspaces classifier",accuracy_score(y_test,y_pred))

Random Subspaces classifier 0.9415


In [ ]:
bag.estimators_samples_[0].shape

(8000,)

In [ ]:
bag.estimators_features_[0].shape

(5,)

# Random Patches
- both column and row sampling doing

In [20]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)

In [21]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Patches classifier",accuracy_score(y_test,y_pred))

Random Patches classifier 0.9425


# OOB Score(out of bag sample )

 

In [22]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)

In [23]:
bag.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [24]:
bag.oob_score_

0.9405

In [25]:
y_pred = bag.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))

Accuracy 0.9425


# Bagging Tips

- Bagging generally gives better results than Pasting
- Good results come around the 25% to 50% row sampling mark
- Random patches and subspaces should be used while dealing with high dimensional data
- To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV

# Applying GridSearchCV

In [26]:
from sklearn.model_selection import GridSearchCV

In [27]:
parameters = {
    'n_estimators': [50,100,500], 
    'max_samples': [0.1,0.4,0.7,1.0],
    'bootstrap' : [True,False],
    'max_features' : [0.1,0.4,0.7,1.0]
    }

In [28]:
search = GridSearchCV(BaggingClassifier(), parameters, cv=5)

In [29]:
search.fit(X_train,y_train)

KeyboardInterrupt: 

In [ ]:
search.best_params_
search.best_score_

0.8986249999999998

In [ ]:
search.best_params_

{'bootstrap': True,
 'max_features': 0.7,
 'max_samples': 0.4,
 'n_estimators': 500}